# Testing speed of the tree algorithm

# Implement a simple network

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
sys.path.append('../')

from src_experiment import NeuralNet
from geobin import RegionTree, TreeNode

In [2]:

net = NeuralNet(
    input_size = 20,
    hidden_sizes = [5,5,5],
    num_classes = 4,
)
state_dict = net.state_dict()

In [3]:
tree = RegionTree(state_dict)

tree.build_tree(verbose=True)

Building tree...


Layer 1:   0%|          | 0/32 [00:00<?, ?it/s]


Layer 4: 100%|██████████| 16/16 [00:10<00:00,  1.47it/s]
